In [ ]:
# 출처 : https://betterprogramming.pub/how-to-build-your-own-custom-chatgpt-with-custom-knowledge-base-4e61ad82427e

#!pip install openai
#!pip install llama-index
#!pip install google-auth-oauthlib

In [ ]:
import os
import pickle
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from llama_index import GPTSimpleVectorIndex, download_loader

os.environ['OPENAI_API_KEY'] = 'sk-xxx'

In [ ]:
# Google 문서 API를 활성화하고 Google 콘솔에서 자격 증명을 가져오려면 다음 단계를 따르세요.

'''
1. Google Cloud Console 웹사이트(console.cloud.google.com)로 이동합니다.

2. 아직 만들지 않은 경우 새 프로젝트를 만듭니다. 
상단 내비게이션 바에서 "프로젝트 선택" 드롭다운 메뉴를 클릭하고 "새 프로젝트"를 선택하면 됩니다. 
프롬프트에 따라 프로젝트 이름을 지정하고 연결하려는 조직을 선택합니다.

3.프로젝트가 생성되면 상단 내비게이션 바의 드롭다운 메뉴에서 선택하세요.

4.왼쪽 메뉴에서 "API 및 서비스" 섹션으로 이동하고 페이지 상단에 있는 "+ API 및 서비스 활성화" 버튼을 클릭합니다.

5.검색창에 "Google Docs API"를 검색하고 결과 목록에서 선택합니다.

6.프로젝트에 대한 API를 활성화하려면 "활성화" 버튼을 클릭하십시오.

7.OAuth 동의 화면 메뉴를 클릭하고 앱에 "mychatbot"과 같은 이름을 만들고 지정한 다음 지원 이메일을 입력하고 저장하고 범위를 추가합니다.

8. 이 Google 앱이 아직 승인되지 않았으므로 테스트 사용자도 추가해야 합니다. 자신의 이메일이 될 수 있습니다.

9.그런 다음 API를 사용하려면 프로젝트에 대한 자격 증명을 설정해야 합니다. 
이렇게 하려면 왼쪽 메뉴에서 "자격 증명" 섹션으로 이동하여 "자격 증명 만들기"를 클릭합니다. 
"OAuth 클라이언트 ID"를 선택하고 프롬프트에 따라 자격 증명을 설정합니다.

10.자격 증명이 설정되면 아래 그림과 같이 JSON 파일을 다운로드하여 애플리케이션의 루트에 저장할 수 있습니다.

자격 증명을 설정하면 Python 프로젝트에서 Google Docs API에 액세스할 수 있습니다.
'''

In [ ]:
# Google 문서를 검색하기 위해 Google 계정에 대해 인증하는 데 도움이 되는 함수를 구성
def authorize_gdocs():
    google_oauth2_scopes = [
        "https://www.googleapis.com/auth/documents.readonly"
    ]
    cred = None
    if os.path.exists("token.pickle"):
        with open("token.pickle", 'rb') as token:
            cred = pickle.load(token)
    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            # 자격증명.json 파일명을 다운로드 받은 파일명으로 변경해야함.
            flow = InstalledAppFlow.from_client_secrets_file("client_secret_611880424174-1jobl5s3djcigva5pj0ocvene8av1e03.apps.googleusercontent.com.json", google_oauth2_scopes)
            cred = flow.run_local_server(port=0)
        with open("token.pickle", 'wb') as token:
            pickle.dump(cred, token)

In [ ]:
# 최신 자격 증명을 승인하거나 다운로드하는 함수
authorize_gdocs() 

In [ ]:
# LlamaIndex Google 문서 리더 초기화
GoogleDocsReader = download_loader('GoogleDocsReader') 

In [ ]:
# 인덱싱하려는 Google 문서 목록
gdoc_ids = ['1q5lTGnZDQ74SX1Cj1ucUsagOCMkZOH-Jk8vP0rjTB74'] 

loader = GoogleDocsReader()

In [ ]:
# 로드 gdocs 및 색인
documents = loader.load_data(document_ids=gdoc_ids)
index = GPTSimpleVectorIndex(documents)    

In [ ]:
# 인덱싱을 파일로 저장 하고 불러옴.
index.save_to_disk('index.json')
# Load the index from your saved index.json file
index = GPTSimpleVectorIndex.load_from_disk('index.json')

In [ ]:
# 인덱싱 쿼리함
while True:
    prompt = input("Type prompt...")
    response = index.query(prompt)
    print(response)

In [ ]:
# 명시적으로 다른 LLM 정의하는 방법
from langchain import OpenAI 
from llama_index import LLMPredictor, GPTSimpleVectorIndex, PromptHelper 

# 명시적으로 다른 LLM 정의
 llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name= "text-davinci-003" )) 

# 프롬프트 구성 정의 
# 최대 설정 입력 크기
 max_input_size = 4096 
# 출력 토큰 수 설정
 num_output = 256 
# 최대 청크 중첩 설정
 max_chunk_overlap = 20 
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap) 

index = GPTSimpleVectorIndex( 
    documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper 
)